# **Trabalho 2 - Reconhecimento de Padrões**
### Árvore de Decisão
##### Renan Henrique Cardoso - 379013

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import itertools
import math
import json
import pandas as pd
pd.options.mode.chained_assignment = None  

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### **Upload do csv**

In [2]:
wines = pd.read_csv("https://raw.githubusercontent.com/cardosorrenan/reconhecimentopadroes-ufc/master/wine.csv")
wines.columns = ['Class', 'Alcohol', 'MalicAcid', 'Gray', 'Alcalinity', 
                  'Magnesium', 'TotalsPhenols', 'Flavanoids', 'PhenolsNonFlav', 
                  'Proanthocy', 'IntensityColor', 'Hue', 'OD', 'Proline']

wines = wines.sample(frac=1)          # Embaralha
wines = wines.reset_index(drop=True)  # Limpa indexes
n_fold = 10                           # Define quantidade de validações

### **Rotulação dos sinais (delimitação dos folds)**

In [3]:
wines['fold'] = pd.DataFrame(data=[-1]*wines.shape[0])  # Todas as linhas da coluna 'fold' recebe -1, pois ainda não pertencem a nenhum fold
samples = math.floor(wines.shape[0] / n_fold)           # Quantidade de amostras em cada fold
for fold in list(range(0, n_fold)):                     # Como K=10, cada linha pertencerá a um fold [0, 1, ..., 9] 
  base = fold*samples
  top = fold*samples + samples - 1
  wines.loc[base:top, 'fold'] = fold     # 0-16 será 0-Fold / 17-33 será 1-Fold / ... / 153-169 será 10-Fold
wines = wines.query('fold != -1')        # Perda de 7 amostras 

### **Funções construídas**

In [4]:
def log(x):
  return 0 if (x == 0) else math.log(x)


# Todas as amostras de um determinado atributo são dispostas ordenadamente
# em um vetor para extração das extremidades de cada classe (limiares)
def getThresholds(df, attr):
  thresholds = []
  classes = df['Class'].unique().tolist()
  for item in classes:
    thresholds.append(df.query(f'Class == {item}')[f'{attr}'].iloc[0])
    thresholds.append(df.query(f'Class == {item}')[f'{attr}'].iloc[-1])
  thresholds.sort()
  thresholds = thresholds[1:-1]
  return thresholds


# Retorna a entropia média do conjunto de dados
def getEntropy(df):
  n_samples = df.shape[0]
  entropy = 0
  classes = df['Class'].unique().tolist()
  for item in classes:
    prob = df.query(f'Class == {item}').shape[0]/n_samples
    entropy = entropy - prob * log(prob)
  return entropy 


# Corta o conjunto de dados dado um atributo e um valor
def splitDataframe(df, attr, value):
  part1 = df.query(f'{attr} <= {value}')
  part2 = df.query(f'{attr} > {value}')
  return part1, part2


# Cada atributo consegue gerar 4 ou 2 limiares (3 ou 2 classes, respectivamente), logo,
# é calculado a entropia de cada um e selecionado o que teve melhor resultado (menor entropia),
# posteriormente, é disputado o melhor limiar entre todos os atributos, selecionando assim o 
# melhor atributo para ser realizado o corte.
def treeDecision(wines):
  best_entropy_attrs = pd.DataFrame(columns=['attribute', 'split_point', 'entropy'])
  n_wines = wines.shape[0]
  
  for attribute in wines.columns[1:]: 
    df_class_attr = wines[['Class',f'{attribute}']]
    df_class_attr = df_class_attr.sort_values(['Class', f'{attribute}'], ascending=[True, True])
    thresholds = getThresholds(df_class_attr, attribute)
    for th in thresholds:
      partition1, partition2 = splitDataframe(df_class_attr, attribute, th)
      entropy_part1 = getEntropy(partition1)
      entropy_part2 = getEntropy(partition2)
      entropy_threshold = (partition1.shape[0]/n_wines) * entropy_part1 + (partition2.shape[0]/n_wines) * entropy_part2
      row = { 'attribute': attribute, 'split_point': th, 'entropy': entropy_threshold }
      best_entropy_attrs = best_entropy_attrs.append(row, ignore_index=True)

  best_entropy_attr = best_entropy_attrs.sort_values('entropy').groupby('attribute').first()
  best_entropy = best_entropy_attr.sort_values('entropy').iloc[0]
  partition1, partition2 = splitDataframe(wines, best_entropy.name, best_entropy.split_point)
  return partition1, partition2, best_entropy


# Constrói o modelo Árvore de Decisão do tipo dicionário (dict), onde a 
# cada corte é criado 3 atributos: o ponto de corte, o atributo escolhido
# para realizar o corte e os dois ramos gerados. Esse procedimento é 
# realizado recursivamente até encontrar o fundo, ou seja, onde a entropia seja 0.
# Quando encontrado, a árvore ganhará somente o atributo 'Classe'.
#
#    Arvore = { 
#       attribute: 'atributo selecionado para corte',
#       split_point: 'ponto de corte',
#       branchs: [
#         { Arvore... }, 
#         { Arvore... }, 
#       ]
#     }
#
#
def build_tree(dataset):
  tree = {}
  if (getEntropy(dataset) != 0):
    partition1, partition2, result = treeDecision(dataset)
    del partition1[result.name]
    del partition2[result.name]
    tree['attribute'] = result.name
    tree['split_point'] = result.split_point
    tree['branchs'] = [ build_tree(partition1), build_tree(partition2) ]
  else:
    tree['Class'] = dataset['Class'].unique().tolist()[0]
  return tree


# Aqui a árvore será testada, ou seja, será realizado os devidos cortes antes
# calculados e criado um dataframe com as amostras então devidamente classificadas.
def testing_tree(tree, dataset):
  result = pd.DataFrame(columns=['Class'])
  if not (u'Class' in tree):
    partition1, partition2 = splitDataframe(dataset, tree['attribute'], tree['split_point'])
    del partition1[tree['attribute']]
    del partition2[tree['attribute']]
    sub_tree1 = tree['branchs'][0]
    sub_tree2 = tree['branchs'][1]
    result = result.append(testing_tree(sub_tree1, partition1))
    result = result.append(testing_tree(sub_tree2, partition2))
  else:
    if dataset.shape[0] > 0:
      df = pd.DataFrame(data=dataset, columns=['Class'])
      df['Class'] = tree['Class']
      return df  
  return result


def compare(x, y):
  x = x.sort_index()
  y = y.sort_index()
  truth_array = np.equal(np.array(x), np.array(y))
  return np.sum(truth_array)/len(truth_array) * 100

### **Loop Principal**

In [5]:
%%time

acc_total = 0

for validation in list(range(0, n_fold)):
  training_data = wines.query(f'fold != {validation}')  # Dados de treinamento
  testing_data = wines.query(f'fold == {validation}')   # Dados de teste
  classes = testing_data['Class']                       # Classes para posterior comparação
  del testing_data['Class']
  model = build_tree(training_data)                     # Constrói árvore recursiva
  predict_classes = testing_tree(model, testing_data)   # Extração das classes
  accuracy = compare(classes, predict_classes['Class'])  # Compara com as classes que realmente elas pertencem e retorna a acurácia
  acc_total += accuracy
  print(f'{validation}-Fold: Accuracy {round(accuracy, 2)}%')

print(f'\nTotal: {round(acc_total/n_fold, 2)}% \n\n')

0-Fold: Accuracy 94.12%
1-Fold: Accuracy 94.12%
2-Fold: Accuracy 88.24%
3-Fold: Accuracy 88.24%
4-Fold: Accuracy 82.35%
5-Fold: Accuracy 100.0%
6-Fold: Accuracy 94.12%
7-Fold: Accuracy 82.35%
8-Fold: Accuracy 88.24%
9-Fold: Accuracy 94.12%

Total: 90.59% 


CPU times: user 53.7 s, sys: 213 ms, total: 53.9 s
Wall time: 53.7 s


## **Alguns insights do algoritmo (última validação 9-fold)**

### **Tempo gasto na construção da árvore:**

In [6]:
%%time
model = build_tree(training_data)  

CPU times: user 5.04 s, sys: 12 ms, total: 5.05 s
Wall time: 5.05 s


### **Tempo gasto para classificação:**

In [7]:
%%time
predict_classes = testing_tree(model, testing_data) 

CPU times: user 107 ms, sys: 3 µs, total: 107 ms
Wall time: 109 ms


### **Estrutura da árvore:**
     
     Arvore = { 
       attribute: 'atributo selecionado para corte',
       split_point: 'ponto de corte',
       branchs: [
         { Arvore... }, 
         { Arvore... }, 
       ]
     }

     Quando houver o atributo 'Classe' no objeto Árvore, significa que chegamos a uma folha (amostras podem ser classificadas)

In [8]:
print(json.dumps(model, indent=2))

{
  "attribute": "OD",
  "split_point": 2.11,
  "branchs": [
    {
      "attribute": "Hue",
      "split_point": 0.96,
      "branchs": [
        {
          "attribute": "IntensityColor",
          "split_point": 3.05,
          "branchs": [
            {
              "Class": 2
            },
            {
              "Class": 3
            }
          ]
        },
        {
          "Class": 2
        }
      ]
    },
    {
      "attribute": "Alcohol",
      "split_point": 12.85,
      "branchs": [
        {
          "attribute": "Proline",
          "split_point": 937.0,
          "branchs": [
            {
              "Class": 2
            },
            {
              "Class": 1
            }
          ]
        },
        {
          "attribute": "Proline",
          "split_point": 680.0,
          "branchs": [
            {
              "attribute": "Magnesium",
              "split_point": 94.0,
              "branchs": [
                {
                  "Class"